In [1]:
import sys
import os

sys.path.append(os.path.abspath('..\..'))

import yaml
import pandas as pd

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k

In [2]:
# read config
with open('..\config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
#dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
del df_test

In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

df = df_train[df_train['rating']>=4].copy()
df = df.groupby('user_id')['item_id'].apply(list).tolist()
te = TransactionEncoder()
df = te.fit(df).transform(df)
df = pd.DataFrame(df, columns=te.columns_)

frequent_itemsets = fpgrowth(df, min_support=0.10, use_colnames=True)
del df
frequent_itemsets

,support,itemsets
0,0.502123,(50)
1,0.417197,(100)
2,0.381104,(181)
3,0.358811,(174)
4,0.354565,(127)
...,...,...
2767,0.142251,"(50, 117, 181)"
2768,0.107219,"(50, 117, 174)"
2769,0.104034,"(50, 172, 117)"
2770,0.113588,"(50, 405)"


In [6]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)
rules = rules[rules['antecedents'].apply(lambda x: len(x) == 1)]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
5,(174),"(98, 50)",0.358811,0.261146,0.200637,0.559172,2.141218,1.0,0.106935,1.676057,0.831231,0.478481,0.403362,0.663732
7,(98),"(100, 174)",0.351380,0.183652,0.139066,0.395770,2.155004,1.0,0.074534,1.351056,0.826314,0.351206,0.259838,0.576498
12,(98),"(50, 100, 174)",0.351380,0.153928,0.117834,0.335347,2.178602,1.0,0.063747,1.272954,0.834063,0.304110,0.214426,0.550432
13,(174),"(98, 100, 50)",0.358811,0.157113,0.117834,0.328402,2.090237,1.0,0.061461,1.255048,0.813466,0.296000,0.203218,0.539201
15,(174),"(98, 181)",0.358811,0.194268,0.158174,0.440828,2.269182,1.0,0.088469,1.440940,0.872305,0.400538,0.306008,0.627518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12761,(143),(98),0.141189,0.351380,0.108280,0.766917,2.182586,1.0,0.058669,2.782789,0.630905,0.281768,0.640648,0.537537
12762,(237),(117),0.245223,0.238854,0.123142,0.502165,2.102395,1.0,0.064570,1.528912,0.694711,0.341176,0.345940,0.508860
12763,(117),(237),0.238854,0.245223,0.123142,0.515556,2.102395,1.0,0.064570,1.558026,0.688898,0.341176,0.358162,0.508860
12765,(181),"(50, 117)",0.381104,0.169851,0.142251,0.373259,2.197563,1.0,0.077519,1.324548,0.880520,0.348052,0.245026,0.605380
